<a href="https://colab.research.google.com/github/9-coding/PyTorch/blob/main/03-translation_between_numpy_torch_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensor(numpy / torch / tensorflow) 간 변환

In [14]:
import numpy as np
import torch
import tensorflow as tf

In [2]:
for c in [np, torch, tf]:
  print(c.__name__, c.__version__)

numpy 1.25.2
torch 2.2.1+cu121
tensorflow 2.15.0


# 정리

## tensor 생성
```
t = (1, 2, 3, 4)
a_np = np.array(t)
a_tf = tf.constant(t)
a_torch = torch.tensor(t)
```
## tf → numpy
- `np = np.array(a_tf)`: 서로 영향x
- `np = a_tf.array`: 서로 영향x
- `np = a_tf.numpy()`: 서로 영향x

## torch → numpy
- `np = np.array(a_torch)`: 서로 영향x
- `np = a_torch.array`: 서로 영향(device="cpu"에서만)
- `np = a_torch.numpy().copy()`: 서로 영향x

## numpy → tf
- `tf = tf.constant(a_np)`: 서로 영향x
- `tf = tf.convert_to_tensor(a_np)`: 서로 영향x

## numpy -> torch
- `torch = torch.tensor(a_np)`: 서로 영향x
- `torch = torch.from_numpy(a_np)`: 서로 영향o

## torch -> tf
- `tf = tf.constant(a_torch)`: 서로 영향o
- `tf = tf.convert_to_tensor(a_torch)`: 서로 영향o


# tf, torch → numpy

일종의 새로운 view를 얻는 것으로 볼 것.

### Tensor 생성

In [15]:
t = (1,2,3,4)

a_np = np.array(t)
a_tf = tf.constant(t)
a_torch = torch.tensor(t)

print(a_np)
print(a_torch)
print(a_tf)

[1 2 3 4]
tensor([1, 2, 3, 4])
tf.Tensor([1 2 3 4], shape=(4,), dtype=int32)


## torch → numpy

원래 tensor와 연결된 상태.

한 쪽에서 data를 변경하면 **다른 쪽에도 영향을 줌.**

→ device=”cpu” 한정

copy method를 사용하면 연결되지 않은 상태로 얻을 수 있음.

In [4]:
# torch 값 변환 후 np 확인
b_np = a_torch.numpy()
print(b_np)

b_np[2] = 777
print(b_np)
print(a_torch) # b_np에서 바꾼 값이 a_torch에도 영향.

a_torch[3] = 888
print(b_np) # a_torch에서 바꾼 값이 b_np에도 영향.

[1 2 3 4]
[  1   2 777   4]
tensor([  1,   2, 777,   4])
[  1   2 777 888]


In [5]:
# torch 값 변환 후 np 확인
a_torch = torch.tensor(t) # 초기화
print("a_torch: ", a_torch)
c_np = np.array(a_torch)
print("c_np: ", c_np)

print("\n<c_np[2] = 777>")
c_np[2] = 777
print("c_np: ", c_np)
print("a_torch: ", a_torch) # c_np에서 바꾼 값이 a_torch에 영향x.

print("\n<a_torch[3] = 888>")
c_np[2] = 3
a_torch[3] = 888
print("a_torch: ", a_torch)
print("c_np: ", c_np) # a_torch에서 바꾼 값이 c_np에 영향x.

a_torch:  tensor([1, 2, 3, 4])
c_np:  [1 2 3 4]

<c_np[2] = 777>
c_np:  [  1   2 777   4]
a_torch:  tensor([1, 2, 3, 4])

<a_torch[3] = 888>
a_torch:  tensor([  1,   2,   3, 888])
c_np:  [1 2 3 4]


### copy

In [6]:
a_torch = torch.tensor(t) # a_torch 초기화
print(a_torch)
a_copy = a_torch.numpy().copy()
a_copy[0] = 999
print(a_copy)
print(a_torch) # a_copy에서 바꾼 값이 a_torch에 영향을 주지 않음.

tensor([1, 2, 3, 4])
[999   2   3   4]
tensor([1, 2, 3, 4])


## tensorflow → numpy

tf.constant가 immutable이므로 **서로 영향을 주지 않음**

In [7]:
b_np = a_tf.numpy()
print(b_np)

b_np[2] = 777
print(b_np)
print(a_tf) # b_np에서 바꾼 값이 a_tf에 영향을 주지 않음.

[1 2 3 4]
[  1   2 777   4]
tf.Tensor([1 2 3 4], shape=(4,), dtype=int32)


In [8]:
c_np = np.array(a_tf)
print("c_np: ", c_np)

print("\n<c_np[2] = 777>")
c_np[2] = 777
print("c_np: ", c_np)
print("a_tf: ", a_tf)

c_np:  [1 2 3 4]

<c_np[2] = 777>
c_np:  [  1   2 777   4]
a_tf:  tf.Tensor([1 2 3 4], shape=(4,), dtype=int32)


## numpy → tf, torch

### dtype 주의 사항
    
numpy는 float64를 기본으로 사용하고 pytorch / tensorflow는 float32를 기본으로 사용.
    
→ numpy로부터 tensor 객체를 얻을 경우 float32로 dtype을 변환해야 함.
<br><br>
    

### **Torch**

**torch 변환 후 np 확인**

- `torch.tensor()`: 서로 영향X
- `torch.from_numpy()`: 서로 영향O

**np 변환 후 torch 확인**

- 얻은 방식과 상관없이 서로 영향.

### **TensorFlow**

**tf 변환 후 np 확인**
-> immutable이므로 바꿀 수 없음.

- `tf.constant()`
- `tf.convert_to_tensor()`

**np 변환 후 tf 확인**

- 얻은 방식과 상관없이 서로 영향 없음.
<br><br>

### Tensor 생성

In [9]:
t = (1,2,3,4)

a_np = np.array(t)
a_torch = torch.tensor(a_np)
b_torch = torch.from_numpy(a_np)
a_tf = tf.constant(a_np)
b_tf = tf.convert_to_tensor(a_np)

print(a_np)
print(a_torch)
print(b_torch)
print(a_tf)
print(b_tf)

[1 2 3 4]
tensor([1, 2, 3, 4])
tensor([1, 2, 3, 4])
tf.Tensor([1 2 3 4], shape=(4,), dtype=int64)
tf.Tensor([1 2 3 4], shape=(4,), dtype=int64)


### Torch

In [10]:
# torch 값 변환 후 np 확인
a_torch[2] = 777
print(a_torch)
print(a_np) # torch.tensor로 얻은 a_torch의 값을 바꿔도 a_np에 영향 없음.

b_torch[2] = 777
print(b_torch)
print(a_np) # torch.from_numpy로 얻은 a_torch의 값을 바꾸면 a_np에 영향.

tensor([  1,   2, 777,   4])
[1 2 3 4]
tensor([  1,   2, 777,   4])
[  1   2 777   4]


In [11]:
# np 값 변환 후 torch 확인.
a_np = np.array(t) # 초기화
print(a_np)
a_np[2] = 777
print(a_np)
print(a_torch)
print(b_torch) # 얻은 방식과 상관없이 a_np의 값을 바꾸면 torch에 영향.

[1 2 3 4]
[  1   2 777   4]
tensor([  1,   2, 777,   4])
tensor([  1,   2, 777,   4])


### Tensorflow


In [33]:
print(a_np)
a_np[2] = 777
print(a_np)
print(a_tf)
print(b_tf) # 얻은 방식과 상관없이 a_np의 값을 바꿔도 tf에는 영향 없음.

[1 2 3 4]
[  1   2 777   4]
tf.Tensor([1 2 3 4], shape=(4,), dtype=int32)
tf.Tensor([1 2 3 4], shape=(4,), dtype=int64)


## torch -> tf

In [31]:
a_torch = torch.tensor([1, 2, 3, 4])
print(a_torch)
x = tf.constant(a_torch)
a_torch[2] = 100
print(a_torch)
print(x)

tensor([1, 2, 3, 4])
tensor([  1,   2, 100,   4])
tf.Tensor([  1   2 100   4], shape=(4,), dtype=int64)


In [29]:
a_torch = torch.tensor([1, 2, 3, 4])
print(a_torch)
x = tf.convert_to_tensor(a_torch)
a_torch[2] = 100
print(a_torch)
print(x)

tensor([1, 2, 3, 4])
tensor([  1,   2, 100,   4])
tf.Tensor([  1   2 100   4], shape=(4,), dtype=int64)
